# Single RIS Experiment 

## 🔧 Import Required Libraries
We start by importing Python libraries for:
- **Numerics & plotting**: NumPy, Matplotlib.
- **Sionna RT**: Scene loading, antenna arrays, Tx/Rx setup.
- **TensorFlow**: Signal computations.
- **SciPy**: Interpolation for antenna patterns.


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time
from pathlib import Path 

# Import Sionna RT components
from sionna.rt import load_scene, Transmitter, Receiver, PlanarArray, Camera, RadioMaterial, RIS
import tensorflow as tf

# For link-level simulations
from sionna.channel import cir_to_ofdm_channel, subcarrier_frequencies, OFDMChannel, ApplyOFDMChannel, CIRDataset
from sionna.nr import PUSCHConfig, PUSCHTransmitter, PUSCHReceiver
from sionna.utils import compute_ber, ebnodb2no, PlotBER
from sionna.ofdm import KBestDetector, LinearDetector
from sionna.mimo import StreamManagement
from scipy.interpolate import RegularGridInterpolator


## Load Scene and Define Materials
We load the lab scene (`lab5.xml`) and update its walls with **ITU-based concrete properties**.
A custom `RadioMaterial` is defined with frequency-dependent permittivity and conductivity.


In [ ]:
def concrete_update(frequency_hz):
    f_ghz = frequency_hz / 1e9
    eps_r = 5.24  # Standard concrete permittivity
    sigma_nominal = 0.0462 * tf.pow(f_ghz, 0.7822)
    thickness_scale = 1.0 / 0.1  # 1 meter / 0.1 meter default
    sigma_adjusted = sigma_nominal * thickness_scale
    return eps_r, sigma_adjusted

scene = load_scene("lab5.xml")
scene.frequency = 28e9  # Hz

# Apply updated concrete material
concrete = RadioMaterial(name="itu_concrete_manual",
                         frequency_update_callback=concrete_update)
scene.add(concrete)

obj = scene.get("lab5_flipped")
obj.radio_material = concrete

scene.preview()


## Load Horn Antenna Pattern from `.uan` File
We define helper functions to:
1. Parse `.uan` files.
2. Interpolate radiation patterns.
3. Return a callable pattern compatible with Sionna.


In [ ]:
def _read_uan(path):
    """
    Parse Wi-Insight .uan file and return grids + linear gain patterns.
    """
    path = Path(path).expanduser()
    lines = [ln.strip() for ln in path.open() if ln.strip()]

    # Extract header info
    hdr = {}
    for ln in lines:
        if ln.startswith("#"):
            continue
        tok = ln.split()
        if len(tok) >= 2 and "_" in tok[0]:
            try:
                hdr[tok[0]] = float(tok[1])
            except ValueError:
                pass

    # Define grids
    p0, p1, dp = (hdr.get(k, v) for k, v in [("phi_min",0), ("phi_max",360), ("phi_inc",1)])
    t0, t1, dt = (hdr.get(k, v) for k, v in [("theta_min",0), ("theta_max",180), ("theta_inc",1)])
    phi_grid   = np.arange(p0, p1+1e-9, dp)
    theta_grid = np.arange(t0, t1+1e-9, dt)

    # Parse numeric values
    rows = []
    for ln in lines:
        if ln[0].isdigit() or ln[0] == "-":
            try:
                rows.append(tuple(map(float, ln.split()[:4])))
            except ValueError:
                continue
    rows = np.asarray(rows)
    if rows.size == 0:
        raise ValueError("No numeric rows found in .uan file.")

    theta_vals, phi_vals, Gt_db, Gp_db = rows.T
    n_th, n_ph = len(theta_grid), len(phi_grid)
    Gt, Gp = np.full((n_th, n_ph), -50.0), np.full((n_th, n_ph), -50.0)

    for t, p, gt, gp in zip(theta_vals, phi_vals, Gt_db, Gp_db):
        i, j = int(round((t - t0)/dt)), int(round((p - p0)/dp))
        if 0 <= i < n_th and 0 <= j < n_ph:
            Gt[i, j], Gp[i, j] = gt+20, gp+20

    Gt_lin, Gp_lin = 10**(Gt/10), 10**(Gp/10)
    return theta_grid, phi_grid, Gt_lin, Gp_lin


def make_uan_callable(uan_file, dtype=tf.complex64):
    """Creates a Sionna-compatible antenna pattern callable from a .uan file."""
    θg, φg, Gt_lin, Gp_lin = _read_uan(uan_file)
    itp_t = RegularGridInterpolator((θg, φg), Gt_lin, bounds_error=False, fill_value=0.0)
    itp_p = RegularGridInterpolator((θg, φg), Gp_lin, bounds_error=False, fill_value=0.0)

    def pattern(theta, phi):
        theta = tf.clip_by_value(theta, 0.0, np.pi)
        phi   = tf.math.floormod(phi + np.pi, 2*np.pi) - np.pi
        th_deg = (theta * 180.0 / np.pi).numpy().clip(0, 180)
        ph_deg = ((phi * 180.0 / np.pi) % 360).numpy()

        e_t = np.sqrt(itp_t(np.stack([th_deg, ph_deg], -1)))
        e_p = np.sqrt(itp_p(np.stack([th_deg, ph_deg], -1)))

        e_t = tf.convert_to_tensor(e_t, dtype=dtype.real_dtype)
        e_p = tf.convert_to_tensor(e_p, dtype=dtype.real_dtype)
        e_t = tf.where(tf.math.is_finite(e_t), e_t, 0.0)
        e_p = tf.where(tf.math.is_finite(e_p), e_p, 0.0)

        return tf.complex(e_t, 0.0), tf.complex(e_p, 0.0)

    return pattern


## Define Tx and Rx Antennas
We use the horn pattern loaded from `.uan` and configure **PlanarArray** objects for
both transmitter (Tx) and receiver (Rx).  
Each array has:
- **1×1 element** (single horn)
- **λ/2 spacing** (vertical and horizontal)
- **Vertical polarization**


In [ ]:
horn_pat = make_uan_callable("horn_WI.uan")

scene.tx_array = PlanarArray(
    num_rows=1,
    num_cols=1,
    vertical_spacing=0.5,   # λ
    horizontal_spacing=0.5, # λ
    pattern=horn_pat,
    polarization="V",
    polarization_model=2
)

scene.rx_array = PlanarArray(
    num_rows=1,
    num_cols=1,
    vertical_spacing=0.5,   # λ
    horizontal_spacing=0.5, # λ
    pattern=horn_pat,
    polarization="V",
    polarization_model=2
)


## Step 5: Place Tx, Rx, and RIS
We configure:
- **Tx** at [...]
- **Rx** at [...]
- **RIS** at [...]

The RIS is configured with:
- **52×52 elements**
- **Focusing lens profile** 
- **Fixed amplitude profile to calibrate (0.21)**


In [ ]:
tx_pos  = [...] # <-- Define positions
ris_pos = [...]
rx_pos  = [...]

tx = Transmitter(name="tx", position=tx_pos)
rx = Receiver(name="rx", position=rx_pos)
scene.add(tx)
scene.add(rx)

ris = RIS(name="ris",
          position=ris_pos,
          num_rows=52,
          num_cols=52)
scene.add(ris)

# Apply focusing lens profile
ris.focusing_lens(tx_pos, rx_pos)

# Amplitude profile
amp = tf.constant(0.21,
                  dtype=ris.amplitude_profile.values.dtype,
                  shape=[ris.num_modes, ris.num_rows, ris.num_cols])
ris.amplitude_profile.values = amp

# Orient devices
ris.look_at(rx_pos)
tx.look_at(ris_pos)
rx.look_at(ris_pos)


## Propagation Path Tracing
We compute multi-bounce propagation paths 

In [ ]:
paths_multi = scene.compute_paths(
    max_depth=3,
    los=False,
    diffraction=False,
    num_samples=1e6
)

scene.preview(paths=paths_multi, clip_at=3)


## Step 7: Received Power Calculation
We define a helper `prx()` function that:
1. Extracts channel impulse response (CIR).
2. Computes total received power from all paths.
3. Converts it to **dBm** scale.

In [ ]:
def prx(paths, P_tx_W=1.0, extra_loss_dB=0.0):
    """
    Compute received power for the Tx/Rx pair.

    Parameters
    ----------
    paths        : sionna.rt.Paths
        Result of scene.compute_paths(...)
    P_tx_W       : float
        Radiated Tx power in watts (includes horn gain).
    extra_loss_dB: float
        Additional loss (e.g., cables, connectors). Default = 0 dB.

    Returns
    -------
    Pr_dBm : float
        Received power in dBm.
    """
    h, _ = paths.cir()               # (h, τ) tuple
    h = h[0, 0, :]                   # complex coeffs
    p_w = tf.reduce_sum(tf.abs(h)**2) * P_tx_W
    pr_dBm = 10*np.log10(p_w.numpy()*1e3 + 1e-12) - extra_loss_dB
    return pr_dBm

# Tx power = 100 mW (20 dBm)
P_TX_W = 0.1
rcs_diff = 0   # RCS difference between reflector models
LOSS_DB = 9.5 - rcs_diff

p_focus = prx(scene.compute_paths(max_depth=3), P_TX_W, LOSS_DB)
print(f"Focusing lens: {p_focus:.2f} dBm")


## Inspect RIS Profiles
Inspect the RIS **amplitude** and **phase** profiles to confirm correct initialization.


In [ ]:
print(ris.amplitude_profile.values)
print(ris.phase_profile.values)
